## **Disclaimer:**
As mentioned in the paper the inference is quite expensive, running this locally was impractical for me. For this reason I tryed this workaround, creating a notebook that I would have been able to run on Google Colab, forking the repository inserting this notebook and doing some changes to the paths on the python files to make everything work in Google Colab.

This notebook contains a slightly modified version of uni-ddim-inversion.py and sim.py.

# Required libraries and imports

In [ ]:
!git clone https://github.com/mustafa-dal/ZeroFake-Mod.git

Cloning into 'ZeroFake-Mod'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 205 (delta 102), reused 49 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (205/205), 15.72 MiB | 23.33 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [2]:
import sys
sys.path.append('/content/ZeroFake-Mod')

In [3]:
!pip install fairscale timm natsort scikit-image einops kornia spacy albumentations pudb invisible-watermark imageio imageio-ffmpeg omegaconf test-tube streamlit nltk fasttext sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1

In [4]:
from functools import partial
from pathlib import Path
from typing import Callable, Dict, List, Optional, Tuple, Union

import numpy as np
import PIL
import torch
import torch.nn.functional as nnf

from diffusers import DDIMScheduler, StableDiffusionPipeline

path = "/home/c01zesh/CISPA-projects/meta_transfer-2023/stable-diffusion/stable-diffusion-v1-4"
path = Path(path).expanduser()
# These variable is not used in my case because here they were importing locally stable-diffusion model. The path should correspond to some path in Linux system

from PIL import Image
from torchvision import transforms

import inspect

import torch
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

from diffusers.configuration_utils import FrozenDict
from diffusers.models import AutoencoderKL, UNet2DConditionModel
try:
    from diffusers.pipeline_utils import DiffusionPipeline
except:
    from diffusers.pipelines.pipeline_utils import DiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput
from diffusers.pipelines.stable_diffusion.safety_checker import \
    StableDiffusionSafetyChecker
from diffusers.schedulers import DDIMScheduler,PNDMScheduler, LMSDiscreteScheduler
from diffusers.utils import deprecate, logging, numpy_to_pil
from natsort import natsorted

from PIL import Image
import requests
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from blipmodels import blip_decoder
import spacy

import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import mean_squared_error as MSE

import os
import sys
import argparse

import nltk
from nltk.corpus import brown
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk import download

# Helper functions

In [5]:
# def replace_first_noun(sentence, replacement):
#     doc = nlp(sentence)
#     for token in doc:
#         if token.tag_ in {"NN", "NNS", "NNP", "NNPS"}:
#             return sentence[:token.idx] + replacement + sentence[token.idx + len(token.text):]
#     return sentence
def replace_first_noun(sentence, replacement):
    doc = nlp(sentence)

    first_chunk = None
    for chunk in doc.noun_chunks:
        if first_chunk is None or chunk.start_char < first_chunk.start_char:
            first_chunk = chunk
    # This preliminary part of the function is essential because noun chunks are not sorted by textual order, so it may happen that a noun that is at the end
    # of the phrase is taken

    if first_chunk:
        if first_chunk[0].pos_ == "DET": # If the first token is a determiner (like "the", "a", "an"), preserve it
            noun_start = first_chunk[1].idx # the start is shifted
            noun_end = first_chunk.end_char
            return sentence[:noun_start] + replacement + sentence[noun_end:]
        else:
            return sentence[:first_chunk.start_char] + replacement + sentence[first_chunk.end_char:]

    return sentence

def backward_ddim(x_t, alpha_t: float, alpha_tm1: float, eps_xt):
    """ from noise to image"""
    return (
        alpha_tm1**0.5
        * (
            (alpha_t**-0.5 - alpha_tm1**-0.5) * x_t
            + ((1 / alpha_tm1 - 1) ** 0.5 - (1 / alpha_t - 1) ** 0.5) * eps_xt
        )
        + x_t
    )

def forward_ddim(x_t, alpha_t: float, alpha_tp1: float, eps_xt):
    """ from image to noise, it's the same as backward_ddim"""
    return backward_ddim(x_t, alpha_t, alpha_tp1, eps_xt)

def load_img(path, target_size=512):
    """Load an image, resize and output -1..1"""
    image = Image.open(path).convert("RGB")

    tform = transforms.Compose(
        [
            transforms.Resize((target_size,target_size)), #resizing to 512x512
            transforms.CenterCrop(target_size), #center cropping
            transforms.ToTensor(),
        ]
    )
    image = tform(image)
    return 2.0 * image - 1.0 # converting the image into [-1,1] range

# Redifinition of StableDiffusionPipeline

In [6]:
class StableDiffusionPipeline(DiffusionPipeline):
    def __init__(
        self,
        vae: AutoencoderKL,
        text_encoder: CLIPTextModel,
        tokenizer: CLIPTokenizer,
        unet: UNet2DConditionModel,
        scheduler: Union[DDIMScheduler, PNDMScheduler, LMSDiscreteScheduler],
        #safety_checker: StableDiffusionSafetyChecker = None,
        #feature_extractor: CLIPFeatureExtractor = None
    ):
        super().__init__()

        self.register_modules( # registers the following components in the pipeline
            vae=vae,
            text_encoder=text_encoder,
            tokenizer=tokenizer,
            unet=unet,
            scheduler=scheduler,
        )
        self.forward_diffusion = partial(self.backward_diffusion, reverse_process=True)

    """
    Transform the prompt into text embedding using CLIPTokenizer
    """
    @torch.inference_mode()
    def get_text_embedding(self, prompt):
        text_input_ids = self.tokenizer(
            prompt,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids
        text_embeddings = self.text_encoder(text_input_ids.to(self.device))[0]
        return text_embeddings


    """
    We obtain the latent vector of the input image of the VAE.
    First we retrieve the DiagonalGaussianDistribution (encoding_dist) that it uses it to sample from it or directly from the mean using .mode().
    Then the latents are scaled.
    """
    @torch.inference_mode()
    def get_image_latents(self, image, sample=True, rng_generator=None):
        encoding_dist = self.vae.encode(image).latent_dist # as mentioned in the paper it's a zero-shot approach but we still need to know a VAE and UNet of a LDM
        if sample:
            encoding = encoding_dist.sample(generator=rng_generator)
        else:
            encoding = encoding_dist.mode()
        latents = encoding * 0.18215
        return latents

    """
    Despite the name this function can do both backward and forward DDIM sampling. In our case it is used as inversion and reconstruction exploiting formulas defined above.
    """
    @torch.inference_mode()
    def backward_diffusion(
        self,
        use_old_emb_i=25, # set the inference step from which changes the text embedding to the modified one
        text_embeddings=None,
        old_text_embeddings=None, # not modified prompt
        new_text_embeddings=None, # modified prompt
        latents: Optional[torch.FloatTensor] = None, # initial latent input, x0 if reverse, x_T if reconstruction
        num_inference_steps: int = 50,
        guidance_scale: float = 7.5, # strength of classifier-free guidance
        callback: Optional[Callable[[int, int, torch.FloatTensor], None]] = None,
        callback_steps: Optional[int] = 1,
        reverse_process: True = False,
        **kwargs,
    ):
        """
        Generate image from text prompt and latents
        """
        do_classifier_free_guidance = guidance_scale > 1.0 # enables classifier-free guidance if guidance scale > 1.0
        self.scheduler.set_timesteps(num_inference_steps)
        # initialize values of alphas, based on the number of inference steps, so we separate 1k training steps based on num_inference_steps
        # 1k, if we have 50 as inference steps, we will have [980, 960, ..., 0] for example
        timesteps_tensor = self.scheduler.timesteps.to(self.device)
        latents = latents * self.scheduler.init_noise_sigma
        # scales the input latents to the same noise level on which the model was trained on
        # -> because VAE scales to a distribution, but might not be the same used in training, so we take it from the scheduler to be sure

        if old_text_embeddings is not None and new_text_embeddings is not None: # a bool to be sure to use the modified prompt from a certain inference step
            prompt_to_prompt = True
        else:
            prompt_to_prompt = False


        for i, t in enumerate(self.progress_bar(timesteps_tensor if not reverse_process else reversed(timesteps_tensor))):
        # this is to show a progress bar when we do inversion process
            if prompt_to_prompt:
                if i < use_old_emb_i: # here it replaces the old one with the modified one
                    text_embeddings = old_text_embeddings
                else:
                    text_embeddings = new_text_embeddings

            latent_model_input = (
                torch.cat([latents] * 2) if do_classifier_free_guidance else latents # if we're using cfg, it duplicates the latents
            )
            latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)
            # So it scales the obtained latent based on the timestep. We must match the scale of the latents used during training, otherwise we would obtain garbage predictions
            noise_pred = self.unet(
                latent_model_input, t, encoder_hidden_states=text_embeddings #predicting the noise guided by text embedding
            ).sample

            if do_classifier_free_guidance:
                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2) # it splits the noise prediction into two parts, one for uncod and the other cond
                noise_pred = noise_pred_uncond + guidance_scale * (
                    noise_pred_text - noise_pred_uncond
                )

            prev_timestep = ( # calculate the previous timestep index for DDIM update
                t
                - self.scheduler.config.num_train_timesteps
                // self.scheduler.num_inference_steps
            )
            # the ratio gives the interval between steps in order to retrieve the previous timestep as a subtraction with the current t.
            # num_train_timeteps is the number of diffusion steps used during training of the diffusion model (typically 1k), that defines how gradually noise is added in forward process.
            # During inference you try to do 1k in 50 inference_steps instead. So those 50 steps are spaced within the 1k-step training schedule

            if callback is not None and i % callback_steps == 0:
                callback(i, t, latents)

            # Here we compute alpha_t and alpha_t-1 (this latter one based on prev_timestep)
            alpha_prod_t = self.scheduler.alphas_cumprod[t] # this is the overline_alpha_t, the cumulative product of alphas up to time t
            alpha_prod_t_prev = ( # we calculate overline_alpha_t-1, setting it to the previous timestep (an approximation of what it was)
                self.scheduler.alphas_cumprod[prev_timestep]
                if prev_timestep >= 0
                else self.scheduler.final_alpha_cumprod # or directly to the last one
            )
            if reverse_process: # if we're doing the reverse process we will swap the directions, inverting variable values
                alpha_prod_t, alpha_prod_t_prev = alpha_prod_t_prev, alpha_prod_t
            latents = backward_ddim( # perform reverse/reconstruction process
                x_t=latents,
                alpha_t=alpha_prod_t,
                alpha_tm1=alpha_prod_t_prev,
                eps_xt=noise_pred,
            )
        return latents


    @torch.inference_mode()
    def decode_image(self, latents: torch.FloatTensor, **kwargs) -> List[Image.Image]:
        scaled_latents = 1 / 0.18215 * latents
        image = [
            self.vae.decode(scaled_latents[i : i + 1]).sample for i in range(len(latents))
        ]
        image = torch.cat(image, dim=0)
        return image

    @torch.inference_mode()
    def torch_to_numpy(self, image) -> List[Image.Image]: # this function was missing and giving error
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()
        return image

In [7]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
#pipe.scheduler = DDIMScheduler.from_config(path / "scheduler")
# They were importing locally the scheduler. However it is a zero-shot approach so this should not be something that they have trained and that lead to some perfomance changes.
# The pipe already include its scheduler, so there's not point on importing it.
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

The config attributes {'feature_extractor': ['transformers', 'CLIPImageProcessor'], 'safety_checker': ['stable_diffusion', 'StableDiffusionSafetyChecker']} were passed to StableDiffusionPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'feature_extractor': ['transformers', 'CLIPImageProcessor'], 'safety_checker': ['stable_diffusion', 'StableDiffusionSafetyChecker']} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

# Image folder creation

Change output_folder to change the path on where the reconstructed image will go.

Change images_path to change the path where it is taking the imgs to be tested.

In [8]:
output_folder = Path("/content/ZeroFake-Mod/reconstructed")

In [9]:
images_path = Path("/content/ZeroFake-Mod/imgs")

### Download Chameleon and use only n of them

This following code is used to download Chameleon dataset from external Google Drive link.

In [10]:
!pip install -q gdown

In [11]:
import zipfile
import random

In [ ]:
!gdown 1PSdLtn9671ohECLb7OO19GqU7pSAyGS5 --output ours.zip

'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
zip_path = "ours.zip"
extract_to = images_path
n = 2

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Filter only files in 1_fake or 0_real
    real_files = [f for f in zip_ref.namelist() if "ours/test/0_real/" in f and not f.endswith("/")]

    selected_files = random.sample(real_files, n)

    for file in selected_files:
        zip_ref.extract(file, extract_to)

In [ ]:
real_images_path = images_path / "ours" / "test" / "0_real"
#real_images = natsorted(real_images_path.glob("*.*"))[:20] # to try putting selected_files here instead of [:20] here
real_images = natsorted([images_path / Path(f) for f in selected_files])

In [15]:
images_list_str= [str(x) for x in real_images]

# Loading BLIP model to generate image prompt

In [16]:
index = 0
nlp = spacy.load("en_core_web_sm")
model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
image_size = 512
print("Loading BLIP model...")
blipmodel = blip_decoder(pretrained=model_url, image_size=image_size, vit='base') # Error solved when using generate with this, that can be found in blip.py and med.py
print("BLIP model loaded.")
blipmodel.eval()
blipmodel = blipmodel.cuda()

print("Found", len(images_list_str), "images.")

Loading BLIP model...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.97G/1.97G [00:24<00:00, 86.1MB/s]


reshape position embedding from 196 to 1024
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth
BLIP model loaded.
Found 2 images.


### Most frequent words in english

In [17]:
word_path = Path("/content/ZeroFake-Mod/words/most-common-nouns-english.csv")

In [18]:
import pandas as pd
df = pd.read_csv(word_path)
df.head()

,Word,Plural
0,time,NaN
1,way,NaN
2,year,NaN
3,work,NaN
4,government,NaN


Here i'm filtering the list to remove articles, words that ends with punctuations and words smaller than 3 characters.

In [19]:
df = df['Word'].str.lower()
df = df[df.str.len() > 2] #filter out elements that have len less than 2, because it was also containing articles
df = df[~df.str.contains(r'[^\w\s]', regex=True)] #filter out elements that contain punctuation

In [20]:
import nltk
nltk.download('punkt_tab')
from nltk.corpus import wordnet as wn
download('wordnet')
download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

I notice that the list was contaning many not concrete nouns and I doubted their power on the adversary prompt. For this reason I decided to try to keep only concrete nouns. Using WordNet to discriminate "physical entities" it keeps only "entities that has physical existence".


In [21]:
def is_concrete_noun(word):
    synsets = wn.synsets(word, pos=wn.NOUN)
    for syn in synsets:
        for path in syn.hypernym_paths():
            if any(h.name().startswith("physical_entity") for h in path):
                return True
    return False

In [22]:
df = df[df.apply(is_concrete_noun)]

In [23]:
len(df)

644

In [24]:
noun_list = [noun for noun in df]

### Cosine sim & obtain perturbed prompts

In [25]:
def cosine_similarity(a, b):
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [26]:
def get_perturbed_prompts(sentence): return [replace_first_noun(sentence,noun) for noun in noun_list]

### all-MiniLM-L6-v2 embeddings

https://www.sbert.net/docs/sentence_transformer/usage/semantic_textual_similarity.html

Related papers:
- https://arxiv.org/pdf/1908.10084

In [27]:
from sentence_transformers import SentenceTransformer, util

sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# DDIM Inversion and reconstruction

This code chunk represent the core of the approach, it is calling the manually defined functions of inversion and reconstruction. In fact it is not calling an instantiation of the pipeline, but using methods that we have defined such as forward diffusion and backward diffusion. Then we manually reconstruct from latents the image, that will be later used to understand the results.

In [28]:
for impath in images_list_str:

    img = load_img(impath).unsqueeze(0).to("cuda")
    print(index)

    prompt = blipmodel.generate(img, sample=True, num_beams=3, max_length=40, min_length=5)[0]

    print(prompt)

    candidate_prompts = get_perturbed_prompts(prompt)

    #original_prompt = sentence_embedding(prompt) # to be changed with sentence_transformer.encode(prompt)
    original_prompt = sentence_transformer.encode(prompt)
    min_sim = 1.0
    selected = None
    for adv in candidate_prompts:
      #adv_prompt_word_vector = sentence_embedding(adv) # to be changed with sentence_transformer.encode(adv)
      adv_prompt_word_vector = sentence_transformer.encode(adv)
      #res = cosine_similarity(adv_prompt_word_vector,original_prompt) # to be changed with sentence_transformer.similarities(adv_prompt_word_vector,original_prompt)
      res = util.cos_sim(adv_prompt_word_vector,original_prompt)
      if res < min_sim:
        min_sim = res
        selected = adv
      #print(f"Candidate prompt: {adv} with cosine similarity score {res}")
    prompt = selected

    #prompt = replace_first_noun(prompt, target_word)

    print(prompt)

    text_embeddings = pipe.get_text_embedding(prompt)

    rng_generator=torch.Generator(device=pipe.device).manual_seed(0)

    image_latents = pipe.get_image_latents(img, rng_generator)

    reversed_latents = pipe.forward_diffusion(
        latents=image_latents,
        text_embeddings=text_embeddings,
        guidance_scale=1,
        num_inference_steps=999,
    )

    reconstructed_latents = pipe.backward_diffusion(
        latents=reversed_latents,
        text_embeddings=text_embeddings,
        guidance_scale=1,
        num_inference_steps=20,
    )

    # guidance_scale=1 so we follow the prompt only

    def latents_to_imgs(latents):
        x = pipe.decode_image(latents)
        x = pipe.torch_to_numpy(x)
        x = numpy_to_pil(x)
        return x

    image = latents_to_imgs(reconstructed_latents)[0]

    print(f"Saving image {index} to {os.path.join(output_folder, str(index) + '.png')}")
    image.save(os.path.join(output_folder, str(index) + '.png'), format="PNG")
    index += 1

0
the man playing guitar with an inscription that says'whats so good? '
the thatcher playing guitar with an inscription that says'whats so good? '


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Saving image 0 to /content/ZeroFake-Mod/reconstructed/0.png
1
the sun rising through clouds
the cash rising through clouds


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Saving image 1 to /content/ZeroFake-Mod/reconstructed/1.png


# Analyzing the results

## Fréchet distance implementation

In [ ]:
!pip install frechetdist

In [ ]:
!pip install shapely

In [ ]:
import shapely
from shapely import LineString
line1 = LineString([(0, 0), (100, 0)])
line2 = LineString([(0, 0), (50, 50), (100, 0)])
print(shapely.frechet_distance(line1,line2))

70.71067811865476


In [ ]:
from matplotlib import pyplot as plt

def show_contours_on_image(gray, contours, title="Contours"):
    image_vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(image_vis, contours, -1, (0, 255, 0), 1)
    plt.imshow(cv2.cvtColor(image_vis, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
"""
In the code below, for each contour in the original image, find the closest one in the reconstructed image using Fréchet distance, and average those distances.
"""
def frechet_distance_score(orig, recon):
    edges1 = cv2.Canny(orig, 100, 200)
    edges2 = cv2.Canny(recon, 100, 200)

    #_, edges1 = cv2.threshold(orig, 127, 255, cv2.THRESH_BINARY)
    #_, edges2 = cv2.threshold(recon, 127, 255, cv2.THRESH_BINARY)
    # plt.imshow(edges1, cmap='gray')
    # plt.title('Thresholded Image')
    # plt.axis('off')
    # plt.show()

    contours_orig, _ = cv2.findContours(edges1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours_recon, _ = cv2.findContours(edges2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    show_contours_on_image(orig, contours_orig, "Contours Image 1")
    show_contours_on_image(recon, contours_recon, "Contours Image 2")

    lines1 = [LineString(c.squeeze()) for c in contours_orig if len(c) >= 2] # each numpy array of coordinates has shape of (N,1,2), where 1 is useless so we squeeze
    lines2 = [LineString(c.squeeze()) for c in contours_recon if len(c) >= 2]
    # requires at least 2 points to define a valid line. If a contour has fewer than 2 points it is a single coordinate

    # Compute pairwise Fréchet distances
    distances = []
    for l1 in lines1:
        # Compute minimum distance between l1 and any contour in image2
        min_dist = min(shapely.frechet_distance(l1, l2) for l2 in lines2)
        distances.append(min_dist)

    # Average minimum distances from image1 to image2
    if len(distances) == 0:
        return float('inf')  # No contours found
    return sum(distances) / len(distances)

## Fréchet Inception distance implementation

The Frechet Inception Distance, or FID for short, is a metric for evaluating the quality of generated images and specifically developed to evaluate the performance of generative adversarial networks.

The FID score uses the Inception v3 model. Specifically, the coding layer of the model (the last pooling layer prior to the output classification of images) is used to capture computer-vision-specific features of an input image. We will take high-level representations of the image.

The feature representation of the two images are saved as vector of dim `(2048,)` and then approximated with multivariate gaussian.

$$
FID = ||\mu_X - \mu_Y|| - Tr(\Sigma_X+\Sigma_Y-2\sqrt{\Sigma_X\Sigma_Y})
$$

In [28]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.0 MB/s eta 0:00:00


In [29]:
from torchvision.models.inception import inception_v3
from scipy.linalg import sqrtm
import torch.nn as nn
from torcheval.metrics import FrechetInceptionDistance

In [30]:
inception_model = inception_v3(pretrained=True, transform_input=False).eval()
inception_model.fc = nn.Identity()

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 121MB/s]


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_model.to(device);

In [32]:
def FID_preprocess(img):
  img = cv2.resize(img, (299, 299))
  img_tensor = torch.from_numpy(img).float()
  img_tensor = img_tensor / 255.0
  img_tensor = img_tensor.permute(2, 0, 1)
  img_tensor = img_tensor.unsqueeze(0).to(device)

  return img_tensor

In [33]:
def extract_features(img):
    img_tensor = FID_preprocess(img)

    # FID implemenetation of torcheval already normalize
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(img_tensor.device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(img_tensor.device)
    img_tensor = (img_tensor - mean) / std

    with torch.no_grad():
        features = inception_model(img_tensor)

    return features.squeeze().cpu().numpy()

Since FID is calculated on batches on images, we can at least consider the difference between the two feature vectors (since we have only two images where one is original and the other reconstructed). So it means that $$\mu_X = f_1$$ and $$\mu_Y = f_2$$

In [34]:
def feature_distance(feat1, feat2):
    return np.sum((feat1 - feat2)**2)

In [35]:
fid = FrechetInceptionDistance().to(device)

## Comparison cycle

In [40]:
folder1_path = real_images_path

folder2_path = output_folder

output_file = "testfake.txt"

mean_ssim_score = []
mean_feature_distance_score = []

with open(output_file, "w") as f:
    f.write("Image Filename\tPixel Similarity\n")
    index = 0
    for image_path1, image_path2 in zip(natsorted(Path(folder1_path).glob("*?.*")), natsorted(Path(folder2_path).glob("*?.*"))):

      print(image_path1)
      print(image_path2)
      image1_orig = cv2.imread(str(image_path1))
      image2_orig = cv2.imread(str(image_path2))

      image1 = cv2.resize(image1_orig, (512, 512))
      image2 = cv2.resize(image2_orig, (512, 512))

      gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
      gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

      # frechet_score = frechet_distance_score(gray1,gray2)
      # print(f"Fréchet score: {frechet_score}")

      features1 = extract_features(image1_orig)
      features2 = extract_features(image2_orig)
      feature_dist = feature_distance(features1,features2)
      mean_feature_distance_score.append(feature_dist)
      print("Feature distance score:", feature_dist,"\n")
      f.write(f"Feature distance score of {index}: \t{feature_dist}\n")

      image1_tensor = FID_preprocess(image1_orig)
      image2_tensor = FID_preprocess(image2_orig)
      fid.update(image1_tensor, is_real=True)
      fid.update(image2_tensor, is_real=False)

      ssim_score = ssim(gray1, gray2)

      f.write(f"SSIM score of {index}: \t{ssim_score}\n")
      print("SSIM score:", ssim_score,"\n")
      index+=1
      mean_ssim_score.append(ssim_score)

    fid_score = fid.compute()
    f.write(f"FID score \t{fid_score.item()}\n")
    print("FID score \t", fid_score.item(),"\n")
    f.write(f"Mean feature distance \t{np.mean(mean_feature_distance_score)}\n")
    print(f"Mean feature distance \t{np.mean(mean_feature_distance_score)}\n")
    f.write(f"Mean SSIM score \t{np.mean(mean_ssim_score)}\n")
    print(f"Mean SSIM score \t{np.mean(mean_ssim_score)}\n")

print("Results saved to", output_file)

/content/ZeroFake-Mod/imgs/Chameleon/test/0_real/2ac22c22-13c5-42a2-9d0a-3e9a32fb620d.jpg
/content/ZeroFake-Mod/reconstructed/0.png
Feature distance score: 491.67343 

SSIM score: 0.6636007427951687 

/content/ZeroFake-Mod/imgs/Chameleon/test/0_real/3bd4b754-391e-43d4-9fad-d41b917d5592.jpg
/content/ZeroFake-Mod/reconstructed/1.png
Feature distance score: 160.63606 

SSIM score: 0.7425017003093806 

/content/ZeroFake-Mod/imgs/Chameleon/test/0_real/6b5026cc-0715-4744-81fd-d9a202b47f8b.jpg
/content/ZeroFake-Mod/reconstructed/2.png
Feature distance score: 71.89605 

SSIM score: 0.7966886705346102 

/content/ZeroFake-Mod/imgs/Chameleon/test/0_real/6b86113c-af38-479c-a74a-d3d56b2861d3.jpg
/content/ZeroFake-Mod/reconstructed/3.png
Feature distance score: 57.91069 

SSIM score: 0.8141616872961359 

/content/ZeroFake-Mod/imgs/Chameleon/test/0_real/80e5f25d-cfbd-4a4e-8419-355dfefd51dd.jpg
/content/ZeroFake-Mod/reconstructed/4.png
Feature distance score: 31.690935 

SSIM score: 0.8008884821197751

In case you want to download the folders containing images.

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("reconstructed", 'zip', "/content/ZeroFake-Mod/reconstructed")
files.download("reconstructed.zip")

shutil.make_archive("imgs", 'zip', "/content/ZeroFake-Mod/imgs")
files.download("imgs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Other code

In order to retrieve the exact correspondent real image from COCO2017, the following code was used.

In [ ]:
import urllib.request
import ssl

url = "https://images.cocodataset.org/train2017/000000301684.jpg"

ssl._create_default_https_context = ssl._create_unverified_context

try:
    urllib.request.urlretrieve(url, "000000301684.jpg")
    print("Image downloaded successfully.")
except urllib.error.URLError as e:
    print(f"Error downloading image: {e}")

from PIL import Image
try:
    img = Image.open("000000301684.jpg")
    img.show()
except FileNotFoundError:
    print("Downloaded image file not found.")

Since the real image taken from RAISE is in .NEF extension (that’s NIKON camera extension), the image was converted into a standard format like PNG, otherwise the model performs really bad giving weird results.

In [ ]:
!pip install rawpy

In [ ]:
import rawpy
from PIL import Image
import numpy as np

with rawpy.imread('/content/r0141f0c9t.NEF') as raw:
    rgb = raw.postprocess()

img = Image.fromarray(rgb)
img.save('r0141f0c9t_mod.png', format='PNG')

### Download Chameleon and use only n of them

This following code is used to download Chameleon dataset from external Google Drive link.

In [ ]:
!pip install -q gdown

In [ ]:
import zipfile

In [ ]:
!gdown 1PSdLtn9671ohECLb7OO19GqU7pSAyGS5 --output ours.zip

In [ ]:
zip_path = "ours.zip"
extract_to = "/content/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Filter only files in 0_real
    real_files = [f for f in zip_ref.namelist() if "ours/test/0_real/" in f and not f.endswith("/")]

    for file in real_files[:20]:
        zip_ref.extract(file, extract_to)

In [ ]:
real_images_path = Path("/content/data/ours/test/0_real")
real_images = natsorted(real_images_path.glob("*.*"))[:20]

In [ ]:
import matplotlib.pyplot as plt
img = cv2.imread(str(real_images[15]))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()